In [2]:
library(tidyverse)
library(knitr)
library('kimisc');

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [3]:
vuln <- read_csv("../permanently_saved_results/arjun-vuln-exp/aug_24_gather.csv",
        col_types ="ccfddfc")

In [4]:
head(vuln)

name,commit,solver,CVE,NDeps,Status,Time
<chr>,<chr>,<fct>,<dbl>,<dbl>,<fct>,<chr>
docusign-node-client,423f93e,npm,NA,NA,missing,NA
docusign-node-client,423f93e,audit fix,105.8,52,success,NA
docusign-node-client,423f93e,audit fix force,0.0,67,success,NA
docusign-node-client,423f93e,maxnpm_cve_oldness,NA,NA,missing,NA
ember-component-css,489f23f,npm,NA,NA,missing,NA
ember-component-css,489f23f,audit fix,106.9,240,success,NA


In [5]:
# What does it mean when status is "missing" - and should we only include those that have no missing?
# There are 225/1000 project-versions that have non-NA results: what is missing there?
vulnByProject <- vuln %>%
    replace_na(list(CVE=-1)) %>%
    pivot_wider(names_from=solver, values_from=CVE, names_prefix="CVE.") %>%
    group_by(name,commit) %>%
    summarise(across(starts_with("CVE."), ~sum(.x,na.rm=TRUE))) %>% ungroup() %>%
    filter(`CVE.audit fix` >= 0 & `CVE.audit fix force`>=0 & `CVE.maxnpm_cve_oldness`>=0) 
head(vulnByProject)
nrow(vulnByProject)

`summarise()` has grouped output by 'name'. You can override using the `.groups` argument.



name,commit,CVE.npm,CVE.audit fix,CVE.audit fix force,CVE.maxnpm_cve_oldness
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
accord,1e5a62b,-1,44.4,0,0
accord,2362ca4,-1,57.2,0,0
accord,391b1c0,-1,0.0,0,0
accord,47b5d29,-1,0.0,0,0
accord,53ef3ed,-1,51.9,0,0
acorn-node,11b3621,-1,0.0,0,0


[1] 225

## How many times did MaxNPM come to a different CVE score than NPM audit fix?

In [6]:
breaks <- c(0, 1, 10, 25, 50, 100, 1000)
bucketedVuln <- vulnByProject %>%
    mutate(across(starts_with('CVE'), ~cut_format(.x, breaks=breaks))) %>% # Bucket all CVE columns
    mutate_if(is.factor, as.character) %>%  # Default is these are factors, make them char so easier to handle
    mutate(across(starts_with('CVE'), ~replace(.,is.na(.), '0'))) %>% # If it was NA, it was because it was outside of the range, aka == 0
    group_by(`CVE.audit fix`,`CVE.maxnpm_cve_oldness`) %>% summarise(count=n()) %>%
    # Sum the count for CVE audit fix and add a row for that called 'total'
    bind_rows(group_by(.,`CVE.audit fix`) %>%
        summarise(count=sum(count)) %>%
        mutate(`CVE.maxnpm_cve_oldness`='Total')) %>%
    bind_rows(group_by(.,`CVE.maxnpm_cve_oldness`) %>%
        summarise(count=sum(count)) %>%
        mutate(`CVE.audit fix`='Total'))

# Create a matrix where the columns are for audit fix, rows are for MaxNPM
bucketedVuln %>%
    pivot_wider(names_from=`CVE.audit fix`, values_from=count) %>%
    relocate(`0`, .before=`(   1,   10]`) %>%
    mutate_if(is.numeric,~replace(.,is.na(.), 0)) %>%
    arrange(CVE.maxnpm_cve_oldness) #%>%
        # kable("html", booktabs=TRUE, linesep="", digits=0, escape=FALSE, format.args = list(big.mark = ",",
        #   scientific = FALSE)) %>% as.character() %>% display_html()

`summarise()` has grouped output by 'CVE.audit fix'. You can override using the `.groups` argument.



CVE.maxnpm_cve_oldness,0,"( 1, 10]","( 10, 25]","( 25, 50]","( 50, 100]",Total
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"( 1, 10]",0,25,0,0,0,25
"( 10, 25]",0,0,31,7,0,38
"( 25, 50]",0,0,0,37,0,37
"( 50, 100]",0,0,0,0,28,28
0,92,1,1,1,2,97
Total,92,26,32,45,30,225


## Which were the cases that MaxNPM better?

In [7]:
vulnByProject %>% filter(`CVE.maxnpm_cve_oldness` < `CVE.audit fix`)

name,commit,CVE.npm,CVE.audit fix,CVE.audit fix force,CVE.maxnpm_cve_oldness
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
accord,1e5a62b,-1,44.40,0.00,0.00
accord,2362ca4,-1,57.20,0.00,0.00
accord,53ef3ed,-1,51.90,0.00,0.00
adbkit,7d0ec60,-1,2.00,0.00,0.00
ampersand-sync,fcc58ef,-1,17.30,0.00,0.00
builder2.js,c5711d4,-1,27.95,22.50,20.45
cozy-db,f8b8e6b,-1,28.35,5.45,22.90
imdone-core,c81c011,-1,74.20,0.00,68.90
jsdoc,2c7c282,-1,30.10,0.00,24.80


In [8]:
vulnByProject %>% filter(`CVE.maxnpm_cve_oldness` > `CVE.audit fix force`)

name,commit,CVE.npm,CVE.audit fix,CVE.audit fix force,CVE.maxnpm_cve_oldness
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
alasql,68b3756,-1,29.00,0,29.00
amd-deploy,e8c4ba5,-1,7.50,0,7.50
anx-api,11904f6,-1,36.85,0,36.85
anx-api,31ee471,-1,36.85,0,36.85
aphcore-lib,2b5b486,-1,58.90,0,58.90
aphcore-lib,81fde86,-1,58.90,0,58.90
api-javascript,69d0d2c,-1,36.85,0,36.85
appjs,2ddd14e,-1,5.45,0,5.45
archiver-utils,7698d99,-1,44.40,0,44.40
